In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 100000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '

RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char
#ctable.char_indices
#ctable.chars

{0: ' ',
 1: '-',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b),reverse=True))  
    if key in seen:
        continue
    seen.add(key)
    a,b = sorted((a, b),reverse=True)
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    #print(query)
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 100000


In [8]:
print(questions[:5], expected[:5])

['976-913', '63-39  ', '47-7   ', '51-8   ', '952-4  '] ['63  ', '24  ', '40  ', '43  ', '948 ']


# Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:80000]
train_y = y[:80000]
test_x = x[80000:]
test_y = y[80000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(72000, 7, 12)
(72000, 4, 12)
Validation Data:
(8000, 7, 12)
(8000, 4, 12)
Testing Data:
(20000, 7, 12)
(20000, 4, 12)


In [11]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

input:  [[[False False False False False False False False False  True False
   False]
  [False False False False False False False False  True False False
   False]
  [False False False False False False False False False False  True
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False  True False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [ True False False False False False False False False False False
   False]]

 [[False False False False False False False False  True False False
   False]
  [False False  True False False False False False False False False
   False]
  [False False  True False False False False False False False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False  True False False False False False False
   False]
  [False False False False False False Fal

# Build Model

In [12]:
print('Build model...')

model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [13]:
for iteration in range(50):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 12s 172us/step - loss: 1.7224 - acc: 0.3633 - val_loss: 1.5698 - val_acc: 0.4149
Q 627-30  T 597  ☒ 408 
Q 693-71  T 622  ☒ 666 
Q 90-82   T 8    ☒ 11  
Q 239-34  T 205  ☒ 200 
Q 48-7    T 41   ☒ 1   
Q 36-28   T 8    ☒ 1   
Q 965-862 T 103  ☒ 61  
Q 770-39  T 731  ☒ 766 
Q 942-4   T 938  ☒ 744 
Q 688-62  T 626  ☒ 778 

--------------------------------------------------
Iteration 1
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 10s 143us/step - loss: 1.4665 - acc: 0.4586 - val_loss: 1.3948 - val_acc: 0.4843
Q 969-108 T 861  ☒ 888 
Q 336-95  T 241  ☒ 299 
Q 797-29  T 768  ☑ 768 
Q 144-89  T 55   ☒ 11  
Q 455-65  T 390  ☒ 318 
Q 54-52   T 

72000/72000 [==============================] - 10s 143us/step - loss: 0.1750 - acc: 0.9513 - val_loss: 0.1536 - val_acc: 0.9590
Q 91-79   T 12   ☑ 12  
Q 965-271 T 694  ☑ 694 
Q 400-31  T 369  ☒ 379 
Q 732-261 T 471  ☑ 471 
Q 360-355 T 5    ☒ 1   
Q 926-35  T 891  ☑ 891 
Q 881-17  T 864  ☑ 864 
Q 308-25  T 283  ☑ 283 
Q 500-4   T 496  ☑ 496 
Q 301-34  T 267  ☑ 267 

--------------------------------------------------
Iteration 14
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 10s 143us/step - loss: 0.1399 - acc: 0.9620 - val_loss: 0.1374 - val_acc: 0.9595
Q 397-26  T 371  ☑ 371 
Q 698-371 T 327  ☑ 327 
Q 230-88  T 142  ☒ 133 
Q 470-155 T 315  ☑ 315 
Q 731-50  T 681  ☑ 681 
Q 772-692 T 80   ☑ 80  
Q 922-919 T 3    ☒ 2   
Q 885-796 T 89   ☑ 89  
Q 183-0   T 183  ☑ 183 
Q 387-14  T 373  ☑ 373 

--------------------------------------------------
Iteration 15
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [===

72000/72000 [==============================] - 10s 144us/step - loss: 0.0326 - acc: 0.9921 - val_loss: 0.0357 - val_acc: 0.9912
Q 297-21  T 276  ☑ 276 
Q 922-127 T 795  ☑ 795 
Q 690-39  T 651  ☑ 651 
Q 899-17  T 882  ☑ 882 
Q 950-71  T 879  ☑ 879 
Q 414-14  T 400  ☑ 400 
Q 558-5   T 553  ☑ 553 
Q 722-78  T 644  ☑ 644 
Q 581-246 T 335  ☑ 335 
Q 838-270 T 568  ☑ 568 

--------------------------------------------------
Iteration 28
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 11s 148us/step - loss: 0.0367 - acc: 0.9902 - val_loss: 0.0596 - val_acc: 0.9793
Q 348-192 T 156  ☑ 156 
Q 131-5   T 126  ☑ 126 
Q 417-50  T 367  ☑ 367 
Q 562-529 T 33   ☑ 33  
Q 567-463 T 104  ☑ 104 
Q 370-53  T 317  ☑ 317 
Q 887-41  T 846  ☑ 846 
Q 414-54  T 360  ☑ 360 
Q 920-12  T 908  ☑ 908 
Q 790-67  T 723  ☑ 723 

--------------------------------------------------
Iteration 29
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [===

72000/72000 [==============================] - 11s 152us/step - loss: 0.0180 - acc: 0.9954 - val_loss: 0.0241 - val_acc: 0.9929
Q 498-465 T 33   ☑ 33  
Q 510-409 T 101  ☑ 101 
Q 320-5   T 315  ☑ 315 
Q 811-88  T 723  ☑ 723 
Q 532-75  T 457  ☑ 457 
Q 950-78  T 872  ☑ 872 
Q 735-258 T 477  ☑ 477 
Q 897-38  T 859  ☑ 859 
Q 279-5   T 274  ☑ 274 
Q 582-63  T 519  ☒ 529 

--------------------------------------------------
Iteration 42
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [==============================] - 11s 152us/step - loss: 0.0189 - acc: 0.9953 - val_loss: 0.0163 - val_acc: 0.9957
Q 49-9    T 40   ☑ 40  
Q 477-391 T 86   ☑ 86  
Q 848-834 T 14   ☑ 14  
Q 354-15  T 339  ☑ 339 
Q 876-151 T 725  ☑ 725 
Q 362-98  T 264  ☑ 264 
Q 703-89  T 614  ☑ 614 
Q 585-46  T 539  ☑ 539 
Q 846-38  T 808  ☑ 808 
Q 938-162 T 776  ☑ 776 

--------------------------------------------------
Iteration 43
Train on 72000 samples, validate on 8000 samples
Epoch 1/1
72000/72000 [===

# Testing

In [15]:
print("MSG : Prediction")
#####################################################
## Try to test and evaluate your model ##############
## ex. test_x = ["555+175", "860+7  ", "340+29 "]
## ex. test_y = ["730 ", "867 ", "369 "] 
#####################################################
equalAnswer = 0
predictions = model.predict_classes(test_x)

for i in range(len(test_x)):
    predAnswer = ctable.decode(predictions[i],False)
    correctAnswer = ctable.decode(test_y[i])
    if(predAnswer == correctAnswer):
        equalAnswer+=1        

accuracyAnswer = equalAnswer/len(test_y)
accuracyAnswer

MSG : Prediction


0.97995